In [1]:
import os
import cv2
import numpy as np

from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [2]:
data_path = "KD_data"

output_model = r"KD_classification_model\classification_model"
output_label_binarizer = r"KD_classification_model\label_binarizer"


In [3]:
KD_labels = set(['kill', 'death', 'none'])

print('Images are being loaded...')

path_to_images = list(paths.list_images(data_path))
data = []
labels = []

for images in path_to_images:
    label = images.split(os.path.sep)[-2]
    if label not in KD_labels:
        continue
    image = cv2.imread(images)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224,224))
    
    data.append(image)
    labels.append(label)


print("Done.")

Images are being loaded...
Done.


In [4]:
data = np.array(data)
labels = np.array(labels)

lb = LabelBinarizer()
labels = lb.fit_transform(labels)


In [5]:
(X_train, X_test, Y_train, Y_test) = train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42)

In [6]:
train_Aug = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

valid_Aug = ImageDataGenerator()
mean = np.array([123.68, 116.779, 103.939], dtype="float32")

train_Aug.mean = mean
valid_Aug.mean = mean

In [7]:
# Model Libraries

from keras.applications import ResNet50

from keras.layers import Input
from keras.layers.pooling import AveragePooling2D
from keras.layers.core import Flatten, Dense, Dropout
from keras.models import Model


In [8]:
# Model

base_model = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(224,224,3)))

head_model = base_model.output
head_model = AveragePooling2D(pool_size=(7,7))(head_model)
head_model = Flatten(name="flatten")(head_model)
head_model = Dense(512, activation='relu')(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense(len(lb.classes_), activation='softmax')(head_model)

model = Model(inputs=base_model.input, outputs=head_model)
epochs = 5

for b_m_layers in base_model.layers:
    b_m_layers.trainable = False


C:\Users\HP\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [9]:
from keras.optimizers import SGD

opt = SGD(lr=0.0001, momentum=0.9, decay=1e-4/epochs)

In [10]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [11]:
history = model.fit_generator(
    train_Aug.flow(X_train, Y_train, batch_size=32),
    steps_per_epoch=len(X_train)//32,
    validation_data=valid_Aug.flow(X_test, Y_test),
    validation_steps=len(X_test)//32,
    epochs=epochs
)


Epoch 1/5
115/115 [==============================] - 1772s 15s/step - loss: 1.1619 - accuracy: 0.4392 - val_loss: 0.9589 - val_accuracy: 0.5378
Epoch 2/5
115/115 [==============================] - 1687s 15s/step - loss: 0.9367 - accuracy: 0.5717 - val_loss: 0.9812 - val_accuracy: 0.6324
Epoch 3/5
115/115 [==============================] - 1653s 14s/step - loss: 0.8078 - accuracy: 0.6475 - val_loss: 0.7785 - val_accuracy: 0.6625
Epoch 4/5
115/115 [==============================] - 1821s 16s/step - loss: 0.7461 - accuracy: 0.6910 - val_loss: 0.6199 - val_accuracy: 0.6859
Epoch 5/5
115/115 [==============================] - 1906s 17s/step - loss: 0.6920 - accuracy: 0.7230 - val_loss: 0.9818 - val_accuracy: 0.7051


In [12]:
# Save Model

import pickle

model.save(output_model)
lbinarizer = open(r"KD_classification_model\KD_classification_binarizer.pickle", "wb")
lbinarizer.write(pickle.dumps(lb))
lbinarizer.close()